***Support Vector Machines***
    Using SVM with Python to predict whether a breast cancer tumor is malignant or benign. Exercise uses numpy, pandas, and scikitlearn and utilizes train test split, SVM, SVC,  and GridSearch to identify the best parameters for prediction.

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
%matplotlib inline

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
cancer = load_breast_cancer()

In [4]:
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
print(cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [7]:
cancer['feature_names']

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [6]:
df_feat = pd.DataFrame(cancer['data'],columns=cancer['feature_names'])
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [8]:
df_target = pd.DataFrame(cancer['target'],columns=['Cancer'])

In [17]:
#Train test split

x_train, x_test, y_train, y_test = train_test_split(df_feat,df_target, test_size = 0.20, random_state=4)

# Applying Standard scaler

scaler = StandardScaler()

scaled_xtrain = scaler.fit_transform(x_train)
scaled_xtest = scaler.fit_transform(x_test)

In [25]:
#Train the support vector classifier
model = SVC(random_state=5)
model.fit(scaled_xtrain, y_train)

C:\Users\NimeshsinhDesai\Anaconda3\envs\SelfDrivingCar\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC(random_state=5)

In [26]:
#Predictions and evaluations
y_pred = model.predict(scaled_xtest)
print(f'Test Score: {model.score(scaled_xtest, y_test)}')

confusion_matrix(y_test, y_pred)

Test Score: 0.9736842105263158


array([[34,  0],
       [ 3, 77]], dtype=int64)

In [27]:
model1 = SVC(random_state=30)
model1.fit(x_train, y_train)

C:\Users\NimeshsinhDesai\Anaconda3\envs\SelfDrivingCar\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC(random_state=30)

In [28]:
#Predictions and evaluations
y_pred1 = model1.predict(x_test)
print(f'Test Score: {model1.score(x_test, y_test)}')

confusion_matrix(y_test, y_pred1)

Test Score: 0.868421052631579


array([[28,  6],
       [ 9, 71]], dtype=int64)

***Everything is being classified into a single class! Use Grid Search to search for the best parameters! (Scikitlearn GridSearchCV)***


In [33]:
#Set parameters
param_grid = {'gamma':[1,0.1,0.001,0.0001],'C':[1,10,100,1000], 'kernel':['linear','rbf','sigmoid','poly']}
grid = GridSearchCV(SVC(),param_grid)

In [42]:
grid.fit(scaled_xtrain, y_train.values.ravel())

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001],
                         'kernel': ['linear', 'rbf', 'sigmoid', 'poly']})

In [43]:
#Fit grid (Note to self: first it runs the same loop with cross-validation to find the best parameter combination. Once it has the best combination, it runs fit again on all data passed to fit (without cross-validation), to built a single new model using the best parameter setting.)
param = grid.best_params_
param

{'C': 1000, 'gamma': 0.001, 'kernel': 'sigmoid'}

In [45]:
#Call the best parameters
model_gridCV = SVC(C=1000, gamma=0.001, kernel='sigmoid')
model_gridCV.fit(scaled_xtrain, y_train.values.ravel())

SVC(C=1000, gamma=0.001, kernel='sigmoid')

In [46]:
#print best score
y_pred2 = model_gridCV.predict(scaled_xtest)
print(f'Test Score: {model_gridCV.score(scaled_xtest, y_test)}')


Test Score: 0.9385964912280702


In [47]:
# print the confusion matrix

confusion_matrix(y_test, y_pred2)

array([[34,  0],
       [ 7, 73]], dtype=int64)

# Discussion of results